## Import

In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

## Data Load

In [2]:
data = pd.read_csv('./train.csv')

# person_id 컬럼 제거
X_train = data.drop(['person_id', 'login'], axis=1)
y_train = data['login']

In [ ]:
# n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
# n_estimators : 10 ~ 1000 사이의 양의 정수
# criterion :  'gini', 'entropy'. 'gini'는 진니 불순도를, 'entropy'는 정보 이득
# max_depth : None 또는 양의 정수
# min_samples_split : 2 이상의 정수 또는 0과 1 사이의 실수
# min_samples_leaf : 1 이상의 정수 또는 0과 0.5 사이의 실수
# min_weight_fraction_leaf :  0.0에서 0.5 사이의 실수
# max_features :  'auto', 'sqrt', 'log2' 또는 양의 정수/실수
# max_leaf_nodes :  None 또는 양의 정수
# min_impurity_decrease : 0.0 이상의 실수
# bootstrap : True, False

## Hyperparameters Search

In [3]:
# GridSearchCV를 위한 하이퍼파라미터 설정
param_search_space = {
    'n_estimators': [10, 50, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 30],
    'min_samples_split': [0.1, 0.9, 2, 10],
    'min_samples_leaf': [0.1, 0.4, 1, 4],
    'min_weight_fraction_leaf': [0.0, 0.2, 0.4, 0.5],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 3, 5],
    'min_impurity_decrease': [0, 0.2, 0.4, 0.6],
    'bootstrap': [True, False]
}

# RandomForestClassifier 객체 생성
rf = RandomForestClassifier(random_state=42)

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=rf, param_grid=param_search_space, cv=3, n_jobs=-1, verbose=2, scoring='roc_auc')

# GridSearchCV를 사용한 학습
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 최고 점수 출력
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

Fitting 3 folds for each of 36 candidates, totalling 108 fits


({'max_depth': None,
  'min_samples_leaf': 4,
  'min_samples_split': 2,
  'n_estimators': 50},
 0.7474650215816369)

## Submission

In [4]:
submit = pd.read_csv('./sample_submission.csv')

# 찾은 최적의 파라미터들을 제출 양식에 맞게 제출
for param, value in best_params.items():
    if param in submit.columns:
        submit[param] = value

submit.to_csv('./baseline_submit.csv', index=False)